<a href="https://colab.research.google.com/github/povinsahu1909/Saransh/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from pprint import pprint

In [105]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en import English
import numpy as np
import requests

nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer'))


target_url = "https://firebasestorage.googleapis.com/v0/b/saransh-36252.appspot.com/o/text%2Fnew.txt?alt=media&token=dd2854fc-4356-4df4-a648-cd311dacc99b"

response = requests.get(target_url)
data = response.text
print(data)

Developers. Developers. Developers. Developers. Developers. Developers. Developers. Developers. Developers. Developers. Developers. Developers. Developers. Developers? Yes. You.


In [1]:
text_corpus = '''Google celebrated British illustrator and artist Sir John Tenniel's 200th birth anniversary with a doodle on February 28. An acclaimed Victorian painter, Tenniel is celebrated for his illustrations for Lewis Carroll's Alice's Adventures in Wonderland and Through the Looking-Glass. Tenniel was born in Bayswater, West London in 1820. At the age of 20, Tenniel received a major eye injury and eventually, lost sight in his right eye. From a very early age, Tenniel was appreciated as a humorist and soon after, also cultured his talent for scholarly caricature. His first illustration was for Samuel Carter Hall's The Book of British Ballads in 1842. Eight years later, he joined the historic weekly magazine Punch as a political cartoonist. Lewis Carroll noticed Tenniel's distinct style of work and in 1864, approached the artist to illustrate his book, Alice's Adventures in Wonderland. This association marked Carroll and Tenniel's creative partnership and continued with Through the Looking Glass in 1872. "The result: a series of classic characters, such as Alice and the Cheshire Cat, as depicted in the Doodle artwork's rendition of their iconic meeting-characters who, along with many others, remain beloved by readers of all ages to this day," the Google Doodle page says. After working with Lewis Carroll, Tenniel resumed his work with Punch. For his work, Tenniel also received a knighthood in 1893. Sir John Tenniel died on February 25, 1914. He was 93.'''


In [103]:
#@title Default title text
def summarizer(text, tokenizer, max_sent_in_summary=3):
    document = nlp(text_corpus.replace("\n", ""))
    sentences = [sent.string.strip() for sent in document.sents]
    sentence_organizer = {k:v for v,k in enumerate(sentences)}
    tf_idf_vectorizer = TfidfVectorizer(min_df=2,  max_features=None, 
                                        strip_accents='unicode', 
                                        analyzer='word',
                                        token_pattern=r'\w{1,}',
                                        ngram_range=(1, 3), 
                                        use_idf=1,smooth_idf=1,
                                        sublinear_tf=1,
                                        stop_words = 'english')
    tf_idf_vectorizer.fit(sentences)
    sentence_vectors = tf_idf_vectorizer.transform(sentences)
    sentence_scores = np.array(sentence_vectors.sum(axis=1)).ravel()
    N = max_sent_in_summary
    top_n_sentences = [sentences[ind] for ind in np.argsort(sentence_scores, axis=0)[::-1][:N]]
    mapped_top_n_sentences = [(sentence,sentence_organizer[sentence]) for sentence in top_n_sentences]
    mapped_top_n_sentences = sorted(mapped_top_n_sentences, key = lambda x: x[1])
    ordered_scored_sentences = [element[0] for element in mapped_top_n_sentences]
    summary = " ".join(ordered_scored_sentences)
    return summary

print("Summarizer Result: \n", summarizer(text=text_corpus, tokenizer=nlp, max_sent_in_summary=3))

Summarizer Result: 
 Google celebrated British illustrator and artist Sir John Tenniel's 200th birth anniversary with a doodle on February 28. An acclaimed Victorian painter, Tenniel is celebrated for his illustrations for Lewis Carroll's Alice's Adventures in Wonderland and Through the Looking-Glass. Lewis Carroll noticed Tenniel's distinct style of work and in 1864, approached the artist to illustrate his book, Alice's Adventures in Wonderland.


In [22]:
text_corpus = '''Google celebrated British illustrator and artist Sir John Tenniel's 200th birth anniversary with a doodle on February 28. An acclaimed Victorian painter, Tenniel is celebrated for his illustrations for Lewis Carroll's Alice's Adventures in Wonderland and Through the Looking-Glass. Tenniel was born in Bayswater, West London in 1820. At the age of 20, Tenniel received a major eye injury and eventually, lost sight in his right eye. From a very early age, Tenniel was appreciated as a humorist and soon after, also cultured his talent for scholarly caricature. His first illustration was for Samuel Carter Hall's The Book of British Ballads in 1842. Eight years later, he joined the historic weekly magazine Punch as a political cartoonist. Lewis Carroll noticed Tenniel's distinct style of work and in 1864, approached the artist to illustrate his book, Alice's Adventures in Wonderland. This association marked Carroll and Tenniel's creative partnership and continued with Through the Looking Glass in 1872. "The result: a series of classic characters, such as Alice and the Cheshire Cat, as depicted in the Doodle artwork's rendition of their iconic meeting-characters who, along with many others, remain beloved by readers of all ages to this day," the Google Doodle page says. After working with Lewis Carroll, Tenniel resumed his work with Punch. For his work, Tenniel also received a knighthood in 1893. Sir John Tenniel died on February 25, 1914. He was 93.'''
text = text_corpus
import re

def tokenize(text):
    pattern = re.compile(r'[A-Za-z0-9]+[\w^\']*|[\w^\']*[A-Za-z0-9]+[\w^\']*')
    return pattern.findall(text.lower())

tokens = tokenize(text)

In [23]:
def mapping(tokens):
    word_to_id = {}
    id_to_word = {}
    
    for i, token in enumerate(set(tokens)):
        word_to_id[token] = i
        id_to_word[i] = token
    
    return word_to_id, id_to_word

word_to_id, id_to_word = mapping(tokens)


In [24]:
import numpy as np

np.random.seed(42)


def generate_training_data(tokens, word_to_id, window):
    X = []
    y = []
    n_tokens = len(tokens)
    
    for i in range(n_tokens):
        idx = concat(
            range(max(0, i - window), i), 
            range(i, min(n_tokens, i + window + 1))
        )
        for j in idx:
            if i == j:
                continue
            X.append(one_hot_encode(word_to_id[tokens[i]], len(word_to_id)))
            y.append(one_hot_encode(word_to_id[tokens[j]], len(word_to_id)))
    
    return np.asarray(X), np.asarray(y)


def concat(*iterables):
    for iterable in iterables:
        yield from iterable


def one_hot_encode(id, vocab_size):
    res = [0] * vocab_size
    res[id] = 1
    return res


X, y = generate_training_data(tokens, word_to_id, 2)


In [17]:
X.shape
y.shape


(934, 138)

In [25]:
def init_network(vocab_size, n_embedding):
    model = {
        "w1": np.random.randn(vocab_size, n_embedding),
        "w2": np.random.randn(n_embedding, vocab_size)
    }
    return model


model = init_network(len(word_to_id), 10)


def forward(model, X, return_cache=True):
    cache = {}
    
    cache["a1"] = X @ model["w1"]
    cache["a2"] = cache["a1"] @ model["w2"]
    cache["z"] = softmax(cache["a2"])
    
    if not return_cache:
        return cache["z"]
    return cache


def softmax(X):
    res = []
    for x in X:
        exp = np.exp(x)
        res.append(exp / exp.sum())
    return res


(X @ model["w1"]).shape
(X @ model["w1"] @ model["w2"]).shape

(934, 138)

In [36]:
def backward(model, X, y, alpha):
    cache  = forward(model, X)
    da2 = cache["z"] - y
    dw2 = cache["a1"].T @ da2
    da1 = da2 @ model["w2"].T
    dw1 = X.T @ da1
    assert(dw2.shape == model["w2"].shape)
    assert(dw1.shape == model["w1"].shape)
    model["w1"] -= alpha * dw1
    model["w2"] -= alpha * dw2
    return cross_entropy(cache["z"], y)


def cross_entropy(z, y):
    return - np.sum(np.log(z) * y)


n_iter = 1
learning_rate = 0.1

history = [backward(model, X, y, learning_rate) for _ in range(n_iter)]

learning = one_hot_encode(word_to_id["google"], len(word_to_id))
result = forward(model, [learning], return_cache=False)[0]

print(result)
# count = 0 
# for word in (id_to_word[id] for id in np.argsort(result)[::-1]):
#     count += 1
#     print(word)

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan]


In [27]:
learning = one_hot_encode(word_to_id["google"], len(word_to_id))
result = forward(model, [learning], return_cache=False)[0]
result

array([0.00830634, 0.00498398, 0.00604721, 0.00932442, 0.00201127,
       0.00033416, 0.01329309, 0.01610928, 0.00018259, 0.00232764,
       0.00164796, 0.0133021 , 0.00340891, 0.00161643, 0.00522385,
       0.01314364, 0.02918905, 0.00754457, 0.00140188, 0.00800645,
       0.0045584 , 0.01034707, 0.00554921, 0.00095867, 0.00690523,
       0.00432935, 0.02240316, 0.00055314, 0.00207516, 0.00115766,
       0.00352728, 0.00256382, 0.00022834, 0.00406192, 0.00414238,
       0.01770899, 0.00173012, 0.00124272, 0.00358867, 0.0012539 ,
       0.00480857, 0.03131056, 0.00163916, 0.00203429, 0.00072793,
       0.0149498 , 0.00133755, 0.00027408, 0.00080501, 0.00889127,
       0.01245869, 0.00145731, 0.00694629, 0.00814219, 0.00404688,
       0.00891762, 0.00313511, 0.01538313, 0.00204016, 0.0014208 ,
       0.03282515, 0.0010558 , 0.00092074, 0.04972168, 0.00143392,
       0.0064275 , 0.00229249, 0.01167329, 0.0030048 , 0.00028578,
       0.01239747, 0.00025927, 0.01535333, 0.00187592, 0.00076

In [28]:
ind = np.argpartition(result, -10)[-10:]
result[ind].sum()

0.394043106755817

In [29]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

stopwords = list(STOP_WORDS)
doc = spacy.load('en_core_web_sm')
doc = doc(text_corpus)

# tokens = [token.text for token in doc]
# punctuation = punctuation + '\n'

word_frequencies = {}

# for word in tokens:
#     if word.lower() not in stopwords:
#         if word.lower() not in punctuation:
#             if word not in word_frequencies.keys():
#                 word_frequencies[word] = 1
#             else:
#                 word_frequencies[word] += 1
for word in tokens:
    if word not in word_frequencies.keys():
        word_frequencies[word] = 1
    else:
        word_frequencies[word] += 1


max_frequency = max(word_frequencies.values())

for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_frequency



In [30]:
sentences_tokens = [sent for sent in doc.sents]
# print(len(sentences_tokens))
sentence_organizer = {k:v for v,k in enumerate(sentences_tokens)}
# print(sentence_organizer)

In [31]:
embeddings = {}
for word in word_to_id:
    learning = one_hot_encode(word_to_id[word], len(word_to_id))
    result = forward(model, [learning], return_cache=False)[0]
    ind = np.argpartition(result, -10)[-10:]
    embeddings[word] = result[ind].sum()


In [32]:
sentence_scores = {}
for sent in sentences_tokens:
    for word in sent:
        if word.text.lower() not in punctuation:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word.text.lower()] * embeddings[word.text.lower()]
                else:
                    sentence_scores[sent] += word_frequencies[word.text.lower()] * embeddings[word.text.lower()]
            else:
                continue


In [33]:
sentence_scores

{Google celebrated British illustrator and artist: 1.4158452297917816,
 Sir John Tenniel's 200th birth anniversary with a doodle on February 28.: 2.7626293301415488,
 An acclaimed Victorian painter, Tenniel is celebrated for his illustrations for Lewis Carroll's Alice's Adventures in Wonderland and Through the Looking-Glass.: 6.718945231020269,
 Tenniel was born in Bayswater, West London in 1820.: 3.4389735979241443,
 At the age of 20, Tenniel received a major eye injury and eventually, lost sight in his right eye.: 6.44324770069177,
 From a very early age, Tenniel was appreciated as a humorist and soon after, also cultured his talent for scholarly caricature.: 5.921434730757202,
 His first illustration was for Samuel Carter Hall's The Book of British Ballads in 1842.: 4.717568870721059,
 Eight years later, he joined the historic weekly magazine Punch as a political cartoonist.: 2.7520059152081635,
 Lewis Carroll noticed Tenniel's distinct style of work and in 1864, approached the arti

In [34]:
# top_n_sentences = 
x = sentence_scores
top_n_sentences = [k for k, v in sorted(x.items(), key=lambda item: item[1])][::-1][:len(sentence_scores)//3]

mapped_sentences = [ (sentence, sentence_organizer[sentence]) for sentence in top_n_sentences]
mapped_top_n_sentences = sorted(mapped_sentences, key = lambda x: x[1])
ordered_sentence = [sentence[0] for sentence in mapped_top_n_sentences]
summary = " "
for sent in ordered_sentence:
    summary += sent.text

ordered_sentence

[An acclaimed Victorian painter, Tenniel is celebrated for his illustrations for Lewis Carroll's Alice's Adventures in Wonderland and Through the Looking-Glass.,
 At the age of 20, Tenniel received a major eye injury and eventually, lost sight in his right eye.,
 From a very early age, Tenniel was appreciated as a humorist and soon after, also cultured his talent for scholarly caricature.,
 Lewis Carroll noticed Tenniel's distinct style of work and in 1864, approached the artist to illustrate his book, Alice's Adventures in Wonderland.,
 "The result: a series of classic characters, such as Alice and the Cheshire Cat, as depicted in the Doodle artwork's rendition of their iconic meeting-characters who, along with many others, remain beloved by readers of all ages to this day," the Google Doodle page says.]